**Installing Keras Tuner**

In [3]:
!pip install keras-tuner
from warnings import filterwarnings
filterwarnings('ignore')

**Importing Libraries**

In [20]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
import numpy as np
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

**Importing Data**

In [5]:
cifar=keras.datasets.cifar10
(X_train,y_train),(X_test,y_test)=cifar.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [6]:
#normalizing the data
X_train=X_train/255.0
X_test=X_test/255.0

In [7]:
#finding the number of classes in the dependent variable
n=len(np.unique(y_test))
n

10

In [8]:
#converting the dependent varibale column to categorical(dummy encoding) as the dependent varibale contains 10 classes
y_train=to_categorical(y_train,n)
y_test=to_categorical(y_test,n)

**Importing Hyperparameter tuning Libraries**

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

**Building Model**

In [10]:
def build_model(hp):
    model=Sequential()

    #Buiding the convolutional layer
    model.add(Conv2D(filters=hp.Int('filter_1',min_value=16,max_value=256,step=16),kernel_size=hp.Choice('kernel_size_1',values=[3,5]),activation='relu',input_shape=(32,32,3)))
    
    #adding the second layer
    model.add(Conv2D(filters=hp.Int('filter_2',min_value=16,max_value=256,step=16),kernel_size=hp.Choice('kernel_size_2',values=[3,5]),activation='relu'))
    model.add(MaxPooling2D(pool_size=hp.Choice('pool_2',values=[2,4,6])))

    #Flattening the feature map
    model.add(Flatten())

    #Adding ANN layers 
    model.add(Dense(units=hp.Int('Unit_1',min_value=16,max_value=256,step=16),activation='relu'))
    model.add(Dense(units=hp.Int('Unit_2',min_value=16,max_value=32,step=16),activation='relu'))
   
    #adding the final layer
    model.add(Dense(units=10,activation='softmax'))

    #compiling the model
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

**Searching for the best parameters using RandomSearch**

In [11]:
tuner=RandomSearch(build_model,
                   objective='val_accuracy',
                   max_trials=3
                  )

In [12]:
tuner.search(X_train,y_train,epochs=3,validation_split=0.2,verbose=2)

Trial 3 Complete [00h 00m 30s]
val_accuracy: 0.651199996471405

Best val_accuracy So Far: 0.6656000018119812
Total elapsed time: 00h 01m 29s


In [13]:
#getting the best model as per the trained parameters
model=tuner.get_best_models(num_models=1)[0]

In [14]:
#Checking the model parameters
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 224)       64736     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 224)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3584)              0         
_________________________________________________________________
dense (Dense)                (None, 192)               688320    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                6176      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                3

In [19]:
#fitting the model as per the best parameters
answer=model.fit(X_train,y_train,epochs=20,initial_epoch=3,validation_split=0.2)

Epoch 4/20
1250/1250 [==============================] - 7s 6ms/step - loss: 0.2973 - accuracy: 0.8947 - val_loss: 1.1059 - val_accuracy: 0.7099
Epoch 5/20
1250/1250 [==============================] - 6s 5ms/step - loss: 0.2526 - accuracy: 0.9096 - val_loss: 1.1087 - val_accuracy: 0.7170
Epoch 6/20
1250/1250 [==============================] - 6s 5ms/step - loss: 0.1998 - accuracy: 0.9304 - val_loss: 1.2717 - val_accuracy: 0.7030
Epoch 7/20
1250/1250 [==============================] - 6s 5ms/step - loss: 0.1747 - accuracy: 0.9381 - val_loss: 1.2903 - val_accuracy: 0.7090
Epoch 8/20
1250/1250 [==============================] - 7s 5ms/step - loss: 0.1575 - accuracy: 0.9439 - val_loss: 1.4113 - val_accuracy: 0.7116
Epoch 9/20
1250/1250 [==============================] - 7s 5ms/step - loss: 0.1331 - accuracy: 0.9520 - val_loss: 1.5215 - val_accuracy: 0.7121
Epoch 10/20
1250/1250 [==============================] - 6s 5ms/step - loss: 0.1243 - accuracy: 0.9567 - val_loss: 1.5687 - val_accuracy

In [16]:
#evaluating the model
model.evaluate(X_test,y_test)

313/313 [==============================] - 1s 3ms/step - loss: 1.0143 - accuracy: 0.7108


[1.0143367052078247, 0.7107999920845032]

So by using Keras tuner we did find the best hyperparameters and hence the accuracy of the model increased and the loss decreased.